9. Haciendo uso de sockets, implemente un servidor y un cliente (a modo de ejemplo, se proveen un servidor y un
cliente en lenguaje Python), que permita desde el cliente, enviar un archivo comprimido utilizando el alfabeto
ABCDEFGH, y en el servidor, descomprimir el archivo.

Este programa implementa un **cliente TCP en Python** que comprime texto usando el algoritmo de Huffman antes de enviarlo a un servidor.  
La compresión se realiza para reducir el tamaño de los datos transmitidos.

In [1]:
import socket
import heapq
import os
from collections import defaultdict

La clase `Nodo` representa cada elemento del árbol de Huffman.  
Cada nodo guarda:
- Un **carácter** y su **frecuencia**.
- Referencias a sus nodos **izquierdo** y **derecho**.  

También se define `__lt__` para que los nodos puedan ser comparados dentro de una **cola de prioridad (heap)**.

In [2]:
class Nodo:
    def __init__(self, caracter, frecuencia):
        self.caracter = caracter
        self.frecuencia = frecuencia
        self.izq = None
        self.der = None

    def __lt__(self, otro):
        return self.frecuencia < otro.frecuencia

La función `construir_arbol` recibe un texto y:
1. Calcula la frecuencia de cada carácter.
2. Crea un heap con nodos iniciales.
3. Combina nodos de menor frecuencia hasta obtener un único árbol raíz.

In [3]:
def construir_arbol(texto):
    frecuencia = defaultdict(int)
    for c in texto:
        frecuencia[c] += 1

    heap = [Nodo(c, f) for c, f in frecuencia.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        n1 = heapq.heappop(heap)
        n2 = heapq.heappop(heap)
        combinado = Nodo(None, n1.frecuencia + n2.frecuencia)
        combinado.izq, combinado.der = n1, n2
        heapq.heappush(heap, combinado)

    return heap[0]

La función `generar_codigos` recorre el árbol y asigna:
- `0` cuando baja a la izquierda.  
- `1` cuando baja a la derecha.  

De esta forma se construye una **tabla de códigos binarios** única para cada carácter.

In [4]:
def generar_codigos(nodo, codigo="", tabla={}):
    if nodo is None:
        return
    if nodo.caracter is not None:
        tabla[nodo.caracter] = codigo
    generar_codigos(nodo.izq, codigo + "0", tabla)
    generar_codigos(nodo.der, codigo + "1", tabla)
    return tabla

La función `comprimir`:
1. Construye el árbol de Huffman.
2. Obtiene la tabla de códigos.
3. Reemplaza cada carácter por su código binario, generando el texto comprimido.

In [5]:
def comprimir(texto):
    raiz = construir_arbol(texto)
    tabla = generar_codigos(raiz)
    comprimido = "".join([tabla[c] for c in texto])
    return comprimido, tabla

En esta sección el programa se conecta al **servidor** usando sockets:
- Dirección: `127.0.0.1`  
- Puerto: `5555`  

Luego:
1. Lee el archivo `comprimido.txt`.
2. Comprime el texto con Huffman.
3. Imprime el original, la tabla de códigos y el resultado comprimido.
4. Envía al servidor **la tabla y el texto comprimido** en un solo paquete.

In [6]:
servidor = "127.0.0.1"
puerto = 5555

cliente = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
cliente.connect((servidor, puerto))

# Texto de ejemplo con alfabeto ABCDEFGH
#texto_original = "AAAABBBCCDAAEEFGHAAABBB"
archivo = "comprimido.txt"

with open(archivo, "rb") as f:
    texto_original = f.read()


comprimido, tabla = comprimir(texto_original)

print("[*] Texto original:", texto_original)
print("[*] Tabla de Huffman:", tabla)
print("[*] Texto comprimido:", comprimido)

# Enviar datos: primero la tabla y luego el texto comprimido
paquete = str(tabla).encode("utf-8") + b"\n" + comprimido.encode("utf-8")
cliente.sendall(paquete)

cliente.close()


[*] Texto original: b'AAAABBBCCDAAEEFGHAAABBB'
[*] Tabla de Huffman: {65: '0', 66: '10', 67: '1100', 72: '11010', 68: '11011', 69: '1110', 71: '11110', 70: '11111'}
[*] Texto comprimido: 000010101011001100110110011101110111111111011010000101010
